In [1]:
import numpy as np
import scipy
import sklearn
import xgboost as xgb
import re
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_dataset = []
test_dataset = []

In [ ]:
dtrain = xgb.DMatrix(data, label=label)
dtest = xgb.DMatrix(data, label=label)

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4

param['eval_metric'] = ['auc', 'f1', 'precision', 'recall']




evallist = [(dtest, 'eval'), (dtrain, 'train')]




num_round = 10
bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=10)
